In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
from typing import List
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from crewai import Agent, Task, Crew
from IPython.display import Markdown, display

In [2]:
# Initialize and constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key) > 10:
    print("API key looks good so far.")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far.


In [3]:
# Define agents
analyzer = Agent(
    role="Business Domain Analyzer",
    goal="Analyze and summarize potential synergies and opportunities between two companies.",
    backstory=(
        "You analyze the information about two companies based on their websites and "
        "determine key synergies, potential partnership opportunities, and areas of mutual benefit. "
        "This allows business leaders to quickly understand how the companies complement each other."
    ),
    allow_delegation=False,
    verbose=True
)

In [4]:
planner = Agent(
    role="Partnership Strategy Planner",
    goal="Draft actionable partnership strategies based on the analysis of company domains.",
    backstory=(
        "You use the analysis of two companies to recommend specific, detailed strategies "
        "for forming mutually beneficial partnerships."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
# Define tasks
fetch_and_analyze = Task(
    description=(
        "1. Scrape and analyze the content from the two provided URLs.\n"
        "2. Identify the core domains of each company.\n"
        "3. Summarize their key business focuses, strengths, and weaknesses."
    ),
    expected_output="A detailed breakdown of the business domains and areas of expertise of the two companies.",
    agent=analyzer,
)

In [6]:
strategize = Task(
    description=(
        "1. Based on the domain analysis, craft specific partnership strategies.\n"
        "2. Highlight areas where the companies could work together effectively.\n"
        "3. Provide actionable recommendations for how to initiate the partnership."
    ),
    expected_output="A detailed partnership strategy document.",
    agent=planner,
)

In [7]:
# Helper function to fetch website content
def get_website_content(url: str) -> str:
    """Fetch and return the text content of a website."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text(strip=True)
    except Exception as e:
        print(f"Error fetching website content for {url}: {e}")
        return ""

In [8]:
# Function to analyze partnerships with Markdown display
def analyze_partnership(url1: str, url2: str):
    """Analyze synergies between two companies and display results in Markdown."""
    print("Fetching website content...")
    content1 = get_website_content(url1)
    content2 = get_website_content(url2)

    if not content1 or not content2:
        print("Error: Unable to fetch content for one or both URLs.")
        return

    print("Analyzing synergies...")

    # Define tasks with the fetched content
    fetch_and_analyze = Task(
        description=(
            f"1. Analyze the following company information:\n\n"
            f"Company 1 Content:\n{content1[:500]}...\n\n"
            f"Company 2 Content:\n{content2[:500]}...\n\n"
            "2. Identify the core domains of each company.\n"
            "3. Summarize their key business focuses, strengths, and weaknesses."
        ),
        expected_output="A detailed breakdown of the business domains and areas of expertise of the two companies.",
        agent=analyzer,
    )

    strategize = Task(
        description=(
            "1. Based on the domain analysis, craft specific partnership strategies.\n"
            "2. Highlight areas where the companies could work together effectively.\n"
            "3. Provide actionable recommendations for how to initiate the partnership."
        ),
        expected_output="A detailed partnership strategy document.",
        agent=planner,
    )

    # Create a crew and add the tasks
    crew = Crew(
        agents=[analyzer, planner],
        tasks=[fetch_and_analyze, strategize],
        verbose=2
    )

    # Execute the tasks
    result = crew.kickoff()
    
    # Display results in Markdown
    markdown_content = f"### Partnership Analysis Results\n\n{result}"
    display(Markdown(markdown_content))

In [9]:
if __name__ == "__main__":
    url1 = input("Enter the first company website URL: ")
    url2 = input("Enter the second company website URL: ")
    analyze_partnership(url1, url2)

Enter the first company website URL:  https://www.odoo.com/
Enter the second company website URL:  https://www.cognizant.com/us/en


Fetching website content...
Analyzing synergies...
 [DEBUG]: == Working Agent: Business Domain Analyzer
 [INFO]: == Starting Task: 1. Analyze the following company information:

Company 1 Content:
Open Source ERP and CRM | OdooSkip to ContentOdooMenuSign inTry it freeAppsFinanceAccountingInvoicingExpensesSpreadsheet (BI)DocumentsSignSalesCRMSalesPOS ShopPOS RestaurantSubscriptionsRentalWebsitesWebsite BuildereCommerceBlogForumLive ChateLearningSupply ChainInventoryManufacturingPLMPurchaseMaintenanceQualityHuman ResourcesEmployeesRecruitmentTime OffAppraisalsReferralsFleetMarketingSocial MarketingEmail MarketingSMS MarketingEventsMarketing AutomationSurveysServicesProjectTimesheetsField Se...

Company 2 Content:
Intuition engineered—human insight, superhuman speed | CognizantSkip to main contentSkip to footerCareersNewsEventsInvestorsUS-ENAmericasArgentinaBrazilCanada(En|Fr)Costa RicaEl SalvadorMexicoUnited StatesMiddle EastVisit our Regional WebsiteEuropeBelgium(Fr|Nl|En)DenmarkFinland

### Partnership Analysis Results

Partnership Strategy for Odoo and Cognizant:

1. Collaborative Product Development: Cognizant, with its innovation in technology solutions, could collaborate with Odoo on product development. This could involve improving the quality and user experience of Odoo's ERP and CRM applications. 

2. Cross-Selling Initiatives: Both Odoo and Cognizant could benefit from cross-selling initiatives. Odoo could refer its clients requiring IT services to Cognizant. Similarly, Cognizant could promote Odoo's business software to its customer base.

3. Global Reach and Localization: Cognizant's global operations could be leveraged to expand Odoo's reach in markets where it has a weak presence. In return, Odoo could help Cognizant specialize its broad service offerings by providing business-specific software solutions.

4. Joint Marketing and Events: Both companies could jointly participate in industry events, webinars, and other marketing activities. This would not only increase their brand visibility but also demonstrate their commitment to providing comprehensive business and technology solutions.

Actionable Recommendations:

1. Arrange a meeting between key stakeholders of both companies to discuss the potential of a partnership and to identify areas of collaboration.

2. Develop a pilot project focusing on collaborative product development. This could involve Cognizant providing technological expertise to improve one of Odoo's applications.

3. Establish a cross-referral program where both companies agree to promote each other's services to their clients.

4. Plan a joint marketing event to announce the partnership and showcase the benefits of the combined service offering.

5. Regularly review and adjust the partnership strategy based on the outcomes and feedback from clients.

By leveraging each other's strengths and offsetting weaknesses, Odoo and Cognizant can create a successful partnership that provides comprehensive solutions to their clients while enhancing their brand value and market reach.